In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
sensor_data=pd.read_csv('iot_telemetry_data.csv')

In [3]:
scaler = MinMaxScaler()
continuous_columns = ['co', 'humidity', 'lpg', 'smoke', 'temp']
sensor_data[continuous_columns] = scaler.fit_transform(sensor_data[continuous_columns])

# Encode boolean features as integers
sensor_data['light'] = sensor_data['light'].astype(int)
sensor_data['motion'] = sensor_data['motion'].astype(int)

# Feature Engineering

In [5]:
# Rolling average for temperature and humidity
sensor_data['temp_rollimg_avg']=sensor_data['temp'].rolling(window=5).mean()
sensor_data['humidity_rolling_avg']=sensor_data['humidity'].rolling(window=5).mean()
# Rate of change for `co` levels to detect spikes or anomalies
sensor_data['co_rate_of_change']=sensor_data['co'].diff().fillna(0)

# Model Training and selection

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
X=sensor_data.drop(['temp'],axis=1)
y=sensor_data['temp']

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [9]:
# Check data types of each column
print(X_train.dtypes)


ts                      float64
device                   object
co                      float64
humidity                float64
light                     int32
lpg                     float64
motion                    int32
smoke                   float64
temp_rollimg_avg        float64
humidity_rolling_avg    float64
co_rate_of_change       float64
dtype: object


In [12]:
print(X_train.shape)


(324147, 12)


In [20]:
# Train a RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

# Instantiate the model
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Fit the model to the training data
model.fit(X_train, y_train)


RandomForestRegressor(n_jobs=-1, random_state=42)

# Evaluation

In [24]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Import evaluation metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate MAE, MSE, RMSE, and R2 score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)  # RMSE
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R² Score: {r2}')


Mean Absolute Error (MAE): 0.00127382495838527
Mean Squared Error (MSE): 0.00013269442036859268
Root Mean Squared Error (RMSE): 0.011519306418729934
R² Score: 0.982940266261902


C:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
